# Channel Routing Tutorial

In this notebook, we'll cover the basics routing hillslope discharge time series through a channel network. To learn more about how to simulate hillslope discharge, see the [hillslope discharge tutorial notebook](hillslope_discharge.ipynb). To learn more about how to create a channel network topoolgy and geometry from a digital elevation map, check out the [REW extraction tutorial](network_extraction_tutorial.ipynb).

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import pickle
from datetime import date
import pandas as pd

We'll first need to import some classes to populate the channels of our REWs. The modules that are imported here must include all vadose zone, groundwater zone, and channel models that are specified below. 

In [2]:
import sys
from os.path import dirname
parent_dir = dirname(dirname(os.getcwd()))
sys.path.append(os.path.join(parent_dir,'StreamflowTempModel','3_channel_routing'))

from channel import SimpleChannel, NoChannel

First, we'll need to set up some data structures to hold REW parameters and forcing data. We'll assume that these have already been computed and have been stored in the `model_data` subfolder of the parent folder. We'll then need to set some model parameters related to timescales and simuluation domains. These are both very similar to the set up in the [hillslope discharge tutorial notebook](hillslope_discharge.ipynb), and so we won't go into too much detail here. It's worth noting that we specify different timescales for hillslope discharge and channel routing. This provides us the flexibility to dial down the channel routing time step in order to ensure numerical convergence of the kinematic wave solver, while still allowing a coarser timestep for the more simple hillslope discharge solver.

In [3]:
# These dictionaries contain the all the data we'll need to instantiate 
rew_config = pickle.load( open( os.path.join(parent_dir,'model_data','rew_config.p'), "rb" ) )
climate_group_forcing = pickle.load( open( os.path.join(parent_dir,'model_data','climate_group_forcing.p'), "rb" ) )
model_config = pickle.load( open( os.path.join(parent_dir, 'model_data', 'model_config.p'), 'rb'))
channel_params = pickle.load( open( os.path.join(parent_dir,'model_data','channel_params.p'), "rb" ))
hill_groups = pickle.load( open( os.path.join(parent_dir,'model_data','solved_hillslope_discharge.p'), "rb" ) )

#start/stop dates for running model  
#spinup date is the date after start_date for which we assume model is finished spinning up         
start_date = model_config['start_date']
stop_date = model_config['stop_date']
spinup_date = model_config['spinup_date']
Tmax = model_config['Tmax']
dt = model_config['dt_channel']
resample_freq_channel = model_config['resample_freq_channel']
resample_freq_hillslope = model_config['resample_freq_hillslope']
timestamps_hillslope = pd.date_range(start_date, stop_date, freq=resample_freq_hillslope)
timestamps_channel = pd.date_range(start_date, stop_date, freq=resample_freq_channel)
t = np.linspace(0,Tmax,np.ceil(Tmax/dt)+1)

The `channel_params` file must contain a dictionary whose keys are REW IDs, and whose values are themselves dictionaries. These key-value pairs of these inner dictionaries the attribute names and values required to fully populate the user-specified `Channel` class.

In [ ]:
channel_network = {}
for rew_id in rew_config.keys(): 
    args = rew_config[rew_id].copy()
    args.update(channel_params[rew_id])
    channel_network[rew_id] = args['model'](rew_id=rew_id, **args)

We need to compute each channel's discharge time series, starting from the headwaters and moving downstream. There are many ways we might compute an acceptable ordering of channels, but GRASS GIS has already computed one for us; the Shreve index of a stream is defined as 1 if the stream is at the headwaters, and as the sum of its parents' Shreve indices otherwise. We can therefore simply compute channel flow starting with all channels with Shreve index 1 and work our way forward. 

The workhorse function in channel routing is the `update` method required by any inheritor of the abstract `Channel` class. As in solving any nonlinear ODE, there is always the risk of introducing numerical instability in this update method if the time step is not sufficiently small. In this example, we have included a simple proof of concept testing procedure that raises a flag if the channel would be completely evacuated in a single timestep given the current velocity.

We save discharge data as a dictionary whose keys are REW IDs and whose values are time series; this file is placed in `network_volumetric_discharges` in the `model_data` folder. 


In [ ]:
network_volumetric_discharges = {}

rew_ids = rew_config.keys()
shreves = [rew_config[rew_id]['shreve'] for rew_id in rew_ids]

channelQueue = [rew_id for (shreve,rew_id) in sorted(zip(shreves,rew_ids))]

print "Solve REWs in this order:"
print(channelQueue)

for rew_id in channelQueue:
    print 'Working on REW ' + str(rew_id)
    width = []
    length = []
    shreve  = rew_config[rew_id]['shreve']
    group_id = rew_config[rew_id]['group']
    climate_group_id = group_id[1]
    ppt = np.array(climate_group_forcing[climate_group_id][start_date:stop_date].ppt.resample(resample_freq_channel).ffill())
    hillslope_discharge = pd.DataFrame({'discharge':hill_groups[group_id]['discharge']}, index=hill_groups[group_id].index)
    hillslope_overlandFlow = pd.DataFrame({'overlandFlow':hill_groups[group_id]['overlandFlow']}, index=hill_groups[group_id].index)
    hillslope_discharge['discharge'] = hillslope_discharge['discharge'] + hillslope_overlandFlow['overlandFlow']
    hillslope_volumetric_discharge = np.array(hillslope_discharge[start_date:stop_date].discharge.resample(resample_freq_channel).ffill())*rew_config[rew_id]['area_sqcm']
    volumetric_discharge = np.zeros(np.size(t))
    volumes = np.zeros(np.size(t))
    approx = 0
    
    if shreve == 1:
        up = np.zeros(np.shape(t))
    else:
        upstream_1 = rew_config[rew_id]['prev_str01']
        upstream_2 = rew_config[rew_id]['prev_str02']
        
        vol_1 = network_volumetric_discharges[upstream_1].volumetric_discharge#.resample(resample_freq_channel).ffill()
        vol_2 = network_volumetric_discharges[upstream_2].volumetric_discharge#.resample(resample_freq_channel).ffill()
        
        up = np.array(vol_1 + vol_2)
    
    for i in range(len(t)):
        volumes[i] = channel_network[rew_id].volume
        result = channel_network[rew_id].update(dt, upstream_volumetric_discharge=up[i], hillslope_volumetric_discharge=hillslope_volumetric_discharge[i] , ppt=ppt[i])
        approx = (approx or result)
        volumetric_discharge[i]=channel_network[rew_id].volumetric_discharge
        width.append(channel_network[rew_id].width)
        length.append(channel_network[rew_id].length)

    if approx==1: 
        print '\nWarning: Numerical instability encountered. Consider decreasing timestep size. \nDischarge for REW ' + str(rew_id) + ' had to be approximated for some timesteps. \n'

    network_volumetric_discharges[rew_id]=pd.DataFrame({'volumetric_discharge':volumetric_discharge, 'volumes':volumes, 'width':width, 'length':length}, index=timestamps_channel)


stores = ['volumes']
for rew_id in channelQueue:
    df = pd.DataFrame()
    for col in network_volumetric_discharges[rew_id].columns:
        if col in stores:
            df[col] = network_volumetric_discharges[rew_id][col].resample('D').first()
        else:
            df[col] = network_volumetric_discharges[rew_id][col].resample('D').apply(sum)*dt
            
    network_volumetric_discharges[rew_id]=df
    
pickle.dump( network_volumetric_discharges, open( os.path.join(parent_dir,'model_data','solved_channel_routing.p'), "wb" ) )

In [ ]:
#Find REW with largest total contributing area
calibration_data_filename = 'sf_leggett_runoff.p'
rew_ids = rew_config.keys()
outlet_id = rew_ids[0]
area_max = rew_config[outlet_id]['upstream_area']
for rew_id in rew_ids:
    if rew_config[rew_id]['upstream_area']>area_max:
        outlet_id = rew_id
        area_max = rew_config[rew_id]['upstream_area']
        

discharge = np.array(network_volumetric_discharges[outlet_id].volumetric_discharge/rew_config[outlet_id]['upstream_area'])
data_df = pickle.load( open( os.path.join(parent_dir, 'calibration_data', calibration_data_filename), 'rb'))
data_runoff_df = data_df['runoff'][start_date:stop_date]
data_runoff = np.array(data_df['runoff'][start_date:stop_date])
outflows_df = pd.DataFrame({'data':data_runoff, 'modeled':discharge}, index = data_runoff_df.index)
outflows_df.loc[spinup_date:stop_date].plot(figsize=(10,8))
plt.yscale('log')
plt.xlim(['10-2013','8-2014'])
# plt.ylim([10**-3, 10**1])

In [ ]:
def objective_function(modeled, observed):
    inds = ((modeled != 0) & (observed != 0))
    if np.sum(modeled)<0.01:
        return -9999.0
    elif np.isnan(np.sum(modeled)):
        return -9999.0
    else:
        return 1-np.sum((np.log(observed.loc[inds])-np.log(modeled.loc[inds]))**2)/np.sum((np.log(observed.loc[inds])-np.mean(np.log(observed.loc[inds])))**2)

objective_function(outflows_df.modeled.loc[spinup_date:stop_date], outflows_df.data.loc[spinup_date:stop_date])

In [ ]:
vz = 0
gz = 0
stream = 0
init = 0

cumPpt = 0
cumET = 0

for rew in channelQueue:
    group_id = rew_config[rew]['group']
    climate_group_id = group_id[1]

    area = rew_config[rew]['area_sqcm']

    vz += hill_groups[group_id].storageVZ*area
    gz += hill_groups[group_id].storageGZ*area
    stream += network_volumetric_discharges[rew].volumes

    # must account for precip that falls on both channel AND hillslope
    ppt = climate_group_forcing[climate_group_id][start_date:stop_date].ppt*area 
    ppt_double_counted = ppt*(network_volumetric_discharges[rew].length*network_volumetric_discharges[rew].width)/area
    ppt = ppt + ppt_double_counted
    ET = hill_groups[group_id].ET*area
    cumPpt += ppt.cumsum()
    cumET += ET.cumsum()
    if ~np.isfinite(network_volumetric_discharges[rew].width[0]):
        print('Steady state channel model; no computed width. Ignore mass balance.')


outDis = network_volumetric_discharges[outlet_id].volumetric_discharge
cumOutDis = outDis.cumsum() 



S0 = stream[0] + gz[0] + vz[0]
totalIn = np.array(cumPpt)
S = np.array(gz + vz + stream)
totalOut = np.array(cumET + cumOutDis)
balance = (totalIn[:-1] - totalOut[:-1] + S0)/S[1:]
plt.plot(balance)
plt.ylim([0,1.1])
print(np.max(balance))
print(np.min(balance))

In [ ]:
# check balance of an individual channel 
for rew in rew_config.keys():
    stream = network_volumetric_discharges[rew].volumes
    length = channel_network[rew].length
    width = channel_network[rew].width
    area = length*width
    group_id = rew_config[rew]['group']
    climate_group_id = group_id[1]
    ppt = climate_group_forcing[climate_group_id][start_date:stop_date].ppt*area 
    hillslope_discharge = pd.DataFrame({'discharge':hill_groups[group_id]['discharge']}, index=hill_groups[group_id].index)
    hillslope_overlandFlow = pd.DataFrame({'overlandFlow':hill_groups[group_id]['overlandFlow']}, index=hill_groups[group_id].index)
    hillslope_discharge['discharge'] = hillslope_discharge['discharge'] + hillslope_overlandFlow['overlandFlow']
    hillslope_volumetric_discharge = hillslope_discharge[start_date:stop_date].discharge*rew_config[rew]['area_sqcm']


    outDis = network_volumetric_discharges[rew].volumetric_discharge
    cumOutDis = outDis.cumsum() 

    upstream_1 = rew_config[rew]['prev_str01']
    upstream_2 = rew_config[rew]['prev_str02']
    try:
        vol_1 = network_volumetric_discharges[upstream_1].volumetric_discharge#.resample(resample_freq_channel).ffill()
        vol_2 = network_volumetric_discharges[upstream_2].volumetric_discharge#.resample(resample_freq_channel).ffill()
        inDis = vol_1 + vol_2 + hillslope_volumetric_discharge
    except:
        inDis = hillslope_volumetric_discharge


    cumInDis = inDis.cumsum()

    cumPpt = ppt.cumsum()


    S0 = stream[0]
    totalIn = np.array(cumPpt + cumInDis)
    S =  np.array(stream)
    totalOut = np.array(cumOutDis)
    balance = (totalIn[:-1] - totalOut[:-1] + S0)/S[1:]
    #balance = (totalIn[:-1] - totalOut[:-1] + S0)-S[1:]

    # plt.plot(balance)
#     print rew
    # plt.ylim([0,1.1])
#     print(np.max(balance))
#     print(np.min(balance))